File to check the pickled obects and save them with the existing files if they didn't

In [1]:
import os
import sys
import datetime
import numpy as np
import pickle
import pandas as pd
import time
import logging
import datetime

In [2]:
file_name = r"pir_pickle.pkl"
with open(file_name, 'rb') as file:
    my_vars = pickle.load(file)

print(len(my_vars))

3


In [3]:
data_basics, data_showcase, showcase_missing = my_vars

### Data Basics

In [4]:
len(data_basics), data_basics[0]

(5431,
 ['https://tpirepguide.com/?p=18052',
  '9782K',
  0,
  'March 29th, 2022 |  Category: Carey Eps '])

In [5]:
df = pd.DataFrame(data_basics, columns=['url', 'episodeID', 'errorCode', 'footer'])
# Additional Column
df['urlParam'] = df.url.apply(lambda x: x.split("=")[1])
df['EpsCategory'] = df.footer.apply(lambda x: x.split(":")[1].strip())
df['airDate'] = df.footer.apply(lambda x: x.split("|")[0].strip())
df.head(5)

,url,episodeID,errorCode,footer,urlParam,EpsCategory,airDate
0,https://tpirepguide.com/?p=18052,9782K,0,"March 29th, 2022 | Category: Carey Eps",18052,Carey Eps,"March 29th, 2022"
1,https://tpirepguide.com/?p=18045,9781K,0,"March 28th, 2022 | Category: Carey Eps",18045,Carey Eps,"March 28th, 2022"
2,https://tpirepguide.com/?p=18018,9775K,0,"March 25th, 2022 | Category: Carey Eps",18018,Carey Eps,"March 25th, 2022"
3,https://tpirepguide.com/?p=18007,9774K,0,"March 24th, 2022 | Category: Carey Eps",18007,Carey Eps,"March 24th, 2022"
4,https://tpirepguide.com/?p=17999,9773K,0,"March 23rd, 2022 | Category: Carey Eps",17999,Carey Eps,"March 23rd, 2022"


In [10]:
old_file_path = r"Data\pir_Check.csv"
old_df = pd.read_csv(old_file_path).drop_duplicates()
old_df.head(3)

,url,episodeID,errorCode,footer,urlParam,EpsCategory,airDate
0,https://tpirepguide.com/?p=18052,9782K,0,"March 29th, 2022 | Category: Carey Eps",18052,Carey Eps,"March 29th, 2022"
1,https://tpirepguide.com/?p=18045,9781K,0,"March 28th, 2022 | Category: Carey Eps",18045,Carey Eps,"March 28th, 2022"
2,https://tpirepguide.com/?p=18018,9775K,0,"March 25th, 2022 | Category: Carey Eps",18018,Carey Eps,"March 25th, 2022"


In [11]:
print(len(df), len(old_df))
if len(old_df) > 0:
    df = pd.concat([old_df, df], ignore_index=True)
len(df)

1627 1905


3532

In [7]:
old_file_path = r"Data\pir_Check.csv"
df.to_csv(old_file_path, index=False, header=True)

### Data Showcase

In [8]:
len(data_showcase), data_showcase[0]

(7972,
    SC_ind PrizeValue Contestant Spin1 Spin2 Total   ev1      Comments  \
 0       1      1,340    Maurice    85     0    85  None          None   
 1       2      8,531        Kim  1.00     0  1.00    50  Bonus Spin ►   
 2       3     25,760     Sandra    20    50    70  None          None   
 
   Showcase urlParam  
 0      SS1    9782K  
 1      SS1    9782K  
 2      SS1    9782K  )

In [29]:
error_dfs = []
for i in range(0, len(data_showcase)):
    show_seg = data_showcase[i]
    tot = sum([1 if x in show_seg.columns[j+ 1:] else 0 for j, x in enumerate(show_seg.columns)])
    if tot > 0:
        error_dfs.append(i)

In [34]:
new_showcase = [x for i, x in enumerate(data_showcase) if i not in error_dfs]
new_showcase = pd.concat(new_showcase, ignore_index=True)
new_showcase.head(3)

,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,SO,...,"1,400",Michael,725,Chantel,649,Mark,"1,241",Miko,840,Kimberley
0,550,Scott,40,50,90,0,None,SS1,9212K,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"2,657",Rita,40,80,120.0,0,None,SS1,9212K,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"5,358",Shaun,55,45,1.00,70,Bonus Spin >,SS1,9212K,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
new_showcase.loc[~new_showcase.Chantel.isna()]

,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,SO,...,"1,400",Michael,725,Chantel,649,Mark,"1,241",Miko,840,Kimberley
8472,"11,485",Colby,1.00Bonus,0,1.00Bonus,NaN,0,SS2,6191K,NaN,...,NaN,NaN,80,Spin ►,NaN,NaN,NaN,NaN,NaN,NaN
8473,"18,487",Toni,20,45,65,NaN,0,SS2,6191K,NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
8474,"59,633",Charles,"808,000",0,"808,000",NaN,0,SS2,6191K,NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
old_file_path = r"Data\pir_Content.csv"
old_df = pd.read_csv(old_file_path).drop_duplicates()
old_df.head(3)

,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,SO,...,55,10,65,"4,999",Leonard,85,"1,650",Narineh,30,50
0,"1,340",Maurice,85,0.0,85,0,NaN,SS1,9782K,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"8,531",Kim,1.00,0.0,1.00,50,Bonus Spin ►,SS1,9782K,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"25,760",Sandra,20,50.0,70,0,NaN,SS1,9782K,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
print(len(new_showcase), len(old_df))
if len(old_df) > 0:
    df = pd.concat([old_df, new_showcase], ignore_index=True)
len(df)

9730 1668


11398

In [9]:
df = pd.concat(data_showcase, ignore_index=True)
df.head(3)

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4
0,1,"1,340",Maurice,85,0,85,None,None,SS1,9782K,NaN,NaN,NaN
1,2,"8,531",Kim,1.00,0,1.00,50,Bonus Spin ►,SS1,9782K,NaN,NaN,NaN
2,3,"25,760",Sandra,20,50,70,None,None,SS1,9782K,NaN,NaN,NaN


In [10]:
old_file_path = r"Data\pir_Content.csv"
df.to_csv(old_file_path, index=False, header=True)

### Errors

In [11]:
len(showcase_missing), showcase_missing[0]

(1438, ['https://tpirepguide.com/?p=17628', 'No Contestant lines found'])

In [12]:
df = pd.DataFrame(showcase_missing, columns=['url', 'error'])
df.head(5)

,url,error
0,https://tpirepguide.com/?p=17628,No Contestant lines found
1,https://tpirepguide.com/?p=17623,No Contestant lines found
2,https://tpirepguide.com/?p=17129,No Contestant lines found
3,https://tpirepguide.com/?p=15751,No Contestant lines found
4,https://tpirepguide.com/?p=15240,No Contestant lines found


In [13]:
old_file_path = r"Data\pir_ErrorContent.csv"

In [44]:
old_df = pd.read_csv(old_file_path).drop_duplicates()
old_df.head(3)

,url,reason
0,https://tpirepguide.com/?p=17715,Extra lines: 8
1,https://tpirepguide.com/?p=17628,Extra lines: 6
2,https://tpirepguide.com/?p=17338,Extra lines: 6


In [47]:
print(len(df), len(old_df))
if len(old_df) > 0:
    df = pd.concat([old_df, df], ignore_index=True)
len(df)

20 7


27

In [14]:
df.to_csv(old_file_path, index=False, header=True)